In [1]:
import pandas as pd
import numpy as np
import re
from IPython.display import Image, display, Markdown, HTML
%matplotlib inline

from helper.utils import *

In [2]:
# Make back into .py script

# !jupyter nbconvert --to script read_model.ipynb

In [3]:
infile = "../pyCIMS_model_description.xlsm"

NODE_COL = "Node"
MODEL_SHEET = "Model"
# TODO extract other string constants used below and place them in some constants here

mxl = pd.read_excel(infile, sheet_name=None, header=1)
mdf0 = mxl[MODEL_SHEET].replace({pd.np.nan: None})
mdf0.index += 3  # adjust index to correspond to Excel line numbers (+1: 0 vs 1 origin, +1: header skip, +1: column headers)

In [4]:
node_cols, year_cols = get_node_cols(mdf0)
all_cols = np.concatenate((node_cols, year_cols))
mdf = mdf0.loc[1:,all_cols] # drop irrelevant columns and skip first, empty row

In [5]:
# determine, row ranges for each node def, based on non-empty Node field
node_rows = mdf.Node[~mdf.Node.isnull()] # does not work if node names have been filled in
node_rows.index.name = "Row Number"
last_row = mdf.index[-1]
node_start_ends = zip(node_rows.index,
                      node_rows.index[1:].tolist() + [last_row])

In [6]:
# extract Node DataFrames, at this point still including Technologies
node_dfs = {}
non_node_cols = mdf.columns != NODE_COL
for s, e in node_start_ends:
    node_name = mdf.Node[s]
    node_df = mdf.loc[s+1:e-1]
    node_df = node_df.loc[non_empty_rows(node_df), non_node_cols]
    # mdf.loc[s+1:e-1, "Node"] = node_name
    node_dfs[node_name] = node_df


In [7]:
node_dfs

{'Simulation':          Parameter Source Branch   Unit Value  2000  2005  2010  2015  2020  \
 5  Simulation time   None   None  Years  None  2000  2005  2010  2015  2020   
 6         Currency   None   None    CAD  2015  None  None  None  None  None   
 
    2025  2030  2035  2040  2045  2050  
 5  2025  2030  None  None  None  None  
 6  None  None  None  None  None  None  ,
 'Canada':             Parameter Source          Branch  Unit   Value  2000  2005  2010  \
 9    Service provided   None          Canada  None  Canada  None  None  None   
 10  Service requested   None  Canada.Alberta  None    None     1     1     1   
 
     2015  2020  2025  2030  2035  2040  2045  2050  
 9   None  None  None  None  None  None  None  None  
 10     1     1     1     1     1     1     1     1  ,
 'Alberta':             Parameter   Source                      Branch              Unit  \
 13   Service provided     None              Canada.Alberta              unit   
 14          Attribute  StatC

In [8]:
# ## intermediate output for dev purposes
# for nn, ndf in node_dfs.items():
#     display(Markdown("Node: **{}**".format(nn)))
#     display_df(ndf)

In [9]:
# Extract tech dfs from node df's and rewrite node df without techs
tech_dfs = {}
for nn, ndf in node_dfs.items():
    if any(ndf.Parameter == "Technology"):
        tdfs = {}
        first_row, last_row = ndf.index[0], ndf.index[-1]
        tech_rows = ndf.loc[ndf.Parameter == "Technology"].index
        for trs, tre in zip(tech_rows, tech_rows[1:].tolist()+[last_row]):
            tech_df = ndf.loc[trs:tre-1]
            tech_name = tech_df.iloc[0].Value
            tdfs[tech_name] = tech_df
        tech_dfs[nn] = tdfs
        node_dfs[nn] = ndf.loc[:tech_rows[0]-1]

# Display Nodes and Technologies
`node_dfs` contains a dictionary giving the DataFrames that hold the relevant rows for each node, without Technology info  
`tech_dfs` contains a dictionary for each node naming the technologies and holding the relevant rows in a DF

In [10]:
# display content of entire Model dataframe as separate df's
for nn, ndf in node_dfs.items():
    display(Markdown("Node: **{}**".format(nn)))
    display_df(ndf)
    if nn in tech_dfs:
        for tech_name, tdf in tech_dfs[nn].items():
            display(Markdown("Node / Technology: **{} / {}**".format(nn, tech_name)))
            display_df(tdf)

Node: **Simulation**

,Parameter,Source,Branch,Unit,Value,2000,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050
5,Simulation time,,,Years,,2000,2005,2010,2015,2020,2025,2030,,,,
6,Currency,,,CAD,2015,,,,,,,,,,,


Node: **Canada**

,Parameter,Source,Branch,Unit,Value,2000,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050
9,Service provided,,Canada,,Canada,,,,,,,,,,,
10,Service requested,,Canada.Alberta,,,1,1,1,1,1,1,1,1,1,1,1


Node: **Alberta**

...

,Parameter,Source,Branch,Unit,Value,2000,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050
501,Service provided,,Canada.Alberta.Natural Gas,GJ,Natural Gas,,,,,,,,,,,
502,Retrofit Variance,,,,v,,,,,,,,,,,
503,Discount rate_Social,,,%,r,,,,,,,,,,,
504,Risk rate,,,%,,,,,,,,,,,,
505,Consumer Price Index,,,,CPI,,,,,,,,,,,
506,Price,,,$/GJ,Diesel,,,,,,,,,,,
507,Price,,,$/GJ,Light Fuel Oil,,,,,,,,,,,
508,Price,,,$/GJ,Natural Gas,,,,,,,,,,,
509,Price,,,$/GJ,Electricity,,,,,,,,,,,
510,Price,,,$/GJ,Wood,,,,,,,,,,,


Node: **Solar**

,Parameter,Source,Branch,Unit,Value,2000,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050
520,Service provided,,Canada.Alberta.Solar,GJ,Solar,,,,,,,,,,,


Node: **Wind**

,Parameter,Source,Branch,Unit,Value,2000,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050
523,Service provided,,Canada.Alberta.Wind,GJ,Wind,,,,,,,,,,,


# Display Service node paths and demand connections 

In [11]:
## simple node connectivity output
for nn, ndf in node_dfs.items():
    supply_path = ndf.loc[ndf.Parameter.str.lower() == "service supply", "Branch"]
    if not supply_path.empty:
        display(Markdown("## Node: **{}**".format(nn)))
        display(Markdown("Path: **{}**".format(supply_path.values[0])))
        if any(ndf.Parameter.str.lower() == "service demand"):
            display(Markdown("### Service demand connections"))
            display_df(ndf.loc[ndf.Parameter.str.lower() == "service demand", ["Branch","Value"]])
        if nn in tech_dfs:
            for tech_name, tdf in tech_dfs[nn].items():
                display(Markdown("### Technology service demand connections"))
                display(Markdown("Node / Technology: **{} / {}**".format(nn, tech_name)))
                display_df(tdf.loc[tdf.Parameter.str.lower() == "service demand", ["Branch","Value"]])
#                 display_df(tdf)

In [12]:
nodes = set()
edges = set()

def process_connection(con_path, con_name, what=""):
    if con_name and con_name != con_path.split(".")[-1]:
        display(Markdown("   **{}** '{}': '{}'".format(what, con_name, con_path)))
    else:
        display(Markdown("   **{}**: '{}'".format(what, con_path)))

def process_node_service_demands(ndf, what="", separate_table_per_name=False, func=display_df):
    # instead of just displaying dfs below, we could create an edge for each row
    service_demand_idxs = ndf.Parameter.str.lower() == "service demand"
    if any(service_demand_idxs):
        selected_cols = ["Value", "Branch", "Unit"] + year_cols.tolist()
        service_demand_df = ndf.loc[service_demand_idxs]
        service_demand_names = service_demand_df.Value.unique()
        if any(service_demand_names): # named services
            if not separate_table_per_name:
                sd_df = (service_demand_df.sort_values("Value")[selected_cols]
                         .rename(columns={"Value": what if what else "Service Type"}))
                func(sd_df)
            else: # separate table for each demand name
                for name in service_demand_names:
                    sd_df = (service_demand_df.loc[service_demand_df.Value == name, selected_cols]
                             .rename(columns={"Value": what if what else "Service Type"}))
                    
                    func(sd_df)
        else: # components with unnamed paths only
            func(service_demand_df[["Branch"] + year_cols.tolist()].rename(columns={"Parameter":"Component"}))
#             for con_path in service_demand_df["Branch"].values:
#                 process_connection(con_path, None, what="Component")

def create_edges(df, node_path=None,
                 show_df=False, do_print=True): # use show_df=True for debugging
    if do_print:
        display(Markdown("### Node Path: {}".format(node_path)))
        print("at node '{}'".format(node_path))
    for index, rdf in df.iterrows():
        target_path = rdf["Branch"]
        tech_path = None
        if rdf.index[0] == "Branch":
            target_name = None
        else:
            target_name = rdf.iloc[0]
            if not target_path or target_name != target_path.split(".")[-1]:
                tech_path = ".".join([node_path, target_name])
                if do_print:
                    if target_path:
                        print("    via '{}'".format(tech_path))
                    else:
                        print("    connect '{}'".format(tech_path))
            else:
                if target_path:
                    target_name = None
        if do_print and target_path:
            print("        connect '{}' ".format(target_path))            
        if target_path:
            nodes.add(target_path)
            if tech_path:
                nodes.add(tech_path)
                edges.update([(node_path, tech_path), (tech_path, target_path)])
            else:
                edges.add((node_path, target_path))
        else:
            if tech_path:
                nodes.add(tech_path)
                edges.add((node_path, tech_path))
            else:
                print("WARNING: Missing target path and tech path in node '{}' tech '{}'".format(node_path, target_name))
                display_df(df)
    if show_df:
        display_df(df)

extract_nodes_and_edges = True
if not extract_nodes_and_edges:
    func = lambda df, node_path: display_df(df) # just display
    separate_tables = True
else:
    func = create_edges
    separate_tables = True
for nn, ndf in node_dfs.items():
    supply_path = ndf.loc[ndf.Parameter.str.lower() == "service supply", "Branch"]
    if not supply_path.empty:
        node_path = supply_path.values[0]
        process_node_service_demands(ndf, what="Service Type",
                                     separate_table_per_name=separate_tables,
                                     func=lambda df: func(df, node_path=node_path))
        if nn in tech_dfs:
            all_tech_dfs = pd.concat(tech_dfs[nn].values())
            process_node_service_demands(all_tech_dfs, what="Technology",
                                         separate_table_per_name=separate_tables,
                                         func=lambda df: func(df, node_path=node_path))

## Make graph and display it

In [13]:
import networkx as nx
to_pdot = nx.drawing.nx_pydot.to_pydot
def view_pydot(pdot, width=800):
    plt = Image(pdot.create_png(),
               width=width, unconfined=True)
    display(plt)

In [14]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [15]:
## not very useful layout (don't use this)
#import matplotlib.pyplot as plt
#plt.rcParams['figure.figsize'] = [8, 6]
#nx.draw_networkx(G, font_size=10, labels=dict((nn, nn.split(".")[-1]) for nn in nodes))

In [16]:
#with open("whole-graph.png","wb") as fh:
#    fh.write(to_pdot(G).create_png())

for k, C in enumerate(nx.connected_component_subgraphs(G)):
    pdot = to_pdot(C)
    display(Markdown("### Graph Component {}".format(k)))
    view_pydot(pdot, width=600)